In [2]:
#!/usr/bin/env python
# coding: utf-8

"""Trains a ResNet on the CIFAR10 dataset.
ResNet v1
[a] Deep Residual Learning for Image Recognition
https://arxiv.org/pdf/1512.03385.pdf
ResNet v2
[b] Identity Mappings in Deep Residual Networks
https://arxiv.org/pdf/1603.05027.pdf

本程序调用已经训练好的mnist网络，查看样本在不同batch,不同卷积层中的lid值变化。
"""

from __future__ import print_function
import keras
import math
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.metrics import top_k_categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.datasets import cifar10
from keras.datasets import mnist
import numpy as np
import os
from sklearn.decomposition import PCA
import lid
from lid import LID
import matplotlib.pyplot as plt
from matplotlib import ticker, cm

# Training parameters
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 20
data_augmentation = False
num_classes = 10
# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True
exp_name = 'resNet20_MNIST_BS%d_epochs%d_' % (batch_size, epochs)


# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Input image dimensions.
input_shape = x_train.shape[1:]
if(len(x_train.shape)==3):
    x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
    x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)
    input_shape=(x_train.shape[1],x_train.shape[2],1)

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

train_num,test_num = x_train.shape[0],x_test.shape[0]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)
model=load_model('./saved_models/preTrainedMNIST.h5')
# model.summary()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
y_train shape: (60000, 10)


# Predict Method to get model_intermediate_layer

In [ ]:
# outputs = [layer.output for layer in model.layers]
outputs = []
layers_names=['conv2d_19','conv2d_20','conv2d_21','average_pooling2d_1']
for layer in layers_names:
    outputs.append(model.get_layer(layer).output)
# print(outputs)

model_intermediate_layer = Model(inputs=model.input, outputs=outputs)
outputs_predict = model_intermediate_layer.predict(x_train, batch_size=1024)# 4*60000*各层featureMap.

In [ ]:
outputs_predict_flatten = [[] for layer in range(len(layers_names))]
from progressbar import *
pbar = ProgressBar()
for layer in pbar(range(len(outputs_predict))):
    outputs_predict_flatten = [np.reshape(layer,newshape=(x_train.shape[0],-1)) for layer in outputs_predict]
batch_num=int(x_train.shape[0]/batch_size)
outputs_predict_lid=np.zeros([len(layers_names),x_train.shape[0]])
for i in range(batch_num):
    mask_batch = range(i*batch_size,(i+1)*batch_size)  # 一个样本下标仅出现一次
    for layerIdx in range(len(layers_names)):
        dis = LID(outputs_predict_flatten[layerIdx][mask_batch], outputs_predict_flatten[layerIdx][mask_batch], lid_k)
        outputs_predict_lid[layerIdx][mask_batch] = dis

In [ ]:
# outputs = [layer.output for layer in model.layers]
outputs = []
layers_names=['conv2d_19','conv2d_20','conv2d_21','average_pooling2d_1']
for layer in layers_names:
    outputs.append(model.get_layer(layer).output)
# print(outputs)

model_intermediate_layer = Model(inputs=model.input, outputs=outputs)
outputs_predict = model_intermediate_layer.predict(x_train, batch_size=1024)# 4*60000*各层featureMap.
outputs_predict_flatten = [[] for layer in range(len(layers_names))]
from progressbar import *
pbar = ProgressBar()
for layer in pbar(range(len(outputs_predict))):
    outputs_predict_flatten[layer]=[sample.flatten() for sample in outputs_predict[layer]]
    
# import ipdb; ipdb.set_trace()
layer_sampleIdx_lid = np.empty([len(layers_names),x_train.shape[0]])
batch_num=int(x_train.shape[0]/batch_size)
lid_k = int(batch_size/10)
for i in range(batch_num):
    mask_batch = range(i*batch_size,(i+1)*batch_size)  # 一个样本下标仅出现一次
    for layerIdx in range(len(layers_names)):
        dis = LID(layer_outs_flat[k], layer_outs_flat[k], lid_k)
        
    

[<tf.Tensor 'conv2d_19/BiasAdd:0' shape=(?, 7, 7, 64) dtype=float32>, <tf.Tensor 'conv2d_20/BiasAdd:0' shape=(?, 7, 7, 64) dtype=float32>, <tf.Tensor 'conv2d_21/BiasAdd:0' shape=(?, 7, 7, 64) dtype=float32>, <tf.Tensor 'average_pooling2d_1/AvgPool:0' shape=(?, 1, 1, 64) dtype=float32>]


In [ ]:
lid_k=int(batch_size/10)
    for k in range(len(layers_names)):
        print('LID',k)
        dis = LID(layer_outs_flat[k], layer_outs_flat[k], lid_k)
        for idx in range(batch_size):
            layer_sampleIdx_LID[k][mask_batch[idx]]=dis[idx]
    

In [ ]:
# Predict simplified version,一个样本此时只包含一个lid值。
batch_num=int(x_train.shape[0]/batch_size)
layers_names=['conv2d_19','conv2d_20','conv2d_21','average_pooling2d_1']
layer_sampleIdx_LID=[]
from keras import backend as K
for i in range(batch_num):
    print("batch_num ",i)
    mask_batch = range(i*batch_size,(i+1)*batch_size)  # 一个样本下标仅出现一次
    x_train_batch = x_train[mask_batch]
    inp = model.input  # input placeholder
    outputs = [model.get_layer(layer_name).output for layer_name in layers_names]  # all layer outputs,model.layers[0].output也是占位符，此处生成了一个output列表
    functors = [K.function([inp], [out]) for out in outputs]  # evaluation functions，评估函数list。下标为层数。
    # Testing
    layer_outs = [func([x_train_batch]) for func in functors]
    layer_outs = [layer_out[0] for layer_out in layer_outs]
    layer_outs_flat=[]# 每一层的维度可能不同，切记。
    for k in range(len(layers_names)):
        print('layers_names feature extract',k)
        t=np.empty([len(layer_outs[k]),len(layer_outs[k][0])*len(layer_outs[k][0][0])*len(layer_outs[k][0][0][0])])
        for idx in range(batch_size):
            t[idx]=layer_outs[k][idx].flatten()
        layer_outs_flat.append(t)
        
#   计算当前x_train_batch，使用layer_outs_flat计算在各层中lid的大小
    lid_k=int(batch_size/10)
    for k in range(len(layers_names)):
        print('LID',k)
        dis = LID(layer_outs_flat[k], layer_outs_flat[k], lid_k)
        for idx in range(batch_size):
            layer_sampleIdx_LID[k][mask_batch[idx]]=dis[idx]
# print("layer_sampleIdx_LID finish")

In [ ]:
# plt.boxplot(layer_lid[0],labels=('layer0'))
layers_lid=np.array([[0 for idx in range(x_train.shape[0])] for k in range(len(layers_names))])
print("layers_lid.shape ",layers_lid.shape)
# print(layer_sampleIdx_LID[2])
layer0_lid=np.array(layer_sampleIdx_LID[0])
print("layer0_lid[:50]",layer0_lid[:50])
layer0_lid_noneZero_idx = np.argwhere(layer0_lid>0.01).flatten()
print("layer0_lid_noneZero_idx.shape",layer0_lid_noneZero_idx.shape)
layer0_lid_statistic = layer0_lid[layer0_lid_noneZero_idx]
print("layer0_lid_statistic[:20]",layer0_lid_statistic[:20])
from matplotlib import pyplot as plt
plt.boxplot(layer0_lid_statistic)
plt.show()
# core trained model.
# scores = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', scores[0])